In [5]:
pip install fastapi

  Obtaining dependency information for fastapi from https://files.pythonhosted.org/packages/52/b3/7e4df40e585df024fac2f80d1a2d579c854ac37109675db2b0cc22c0bb9e/fastapi-0.115.6-py3-none-any.whl.metadata
  Obtaining dependency information for starlette<0.42.0,>=0.40.0 from https://files.pythonhosted.org/packages/96/00/2b325970b3060c7cecebab6d295afe763365822b1306a12eeab198f74323/starlette-0.41.3-py3-none-any.whl.metadata
  Obtaining dependency information for typing-extensions>=4.8.0 from https://files.pythonhosted.org/packages/26/9f/ad63fc0248c5379346306f8668cda6e2e2e9c95e01216d2b8ffd9ff037d0/typing_extensions-4.12.2-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/94.8 kB ? eta -:--:--
   ------------------------- -------------- 61.4/94.8 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 94.8/94.8 kB 1.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/73.2 kB ? eta -:--:--
   ---------------------------------------  71.7/73.2 kB

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.1.3 which is incompatible.


In [6]:
pip install "uvicorn[standard]"

  Obtaining dependency information for uvicorn[standard] from https://files.pythonhosted.org/packages/50/c1/2d27b0a15826c2b71dcf6e2f5402181ef85acf439617bb2f1453125ce1f3/uvicorn-0.32.1-py3-none-any.whl.metadata
  Obtaining dependency information for h11>=0.8 from https://files.pythonhosted.org/packages/95/04/ff642e65ad6b90db43e668d70ffb6736436c7ce41fcc549f4e9472234127/h11-0.14.0-py3-none-any.whl.metadata
  Obtaining dependency information for httptools>=0.6.3 from https://files.pythonhosted.org/packages/d0/46/4d8e7ba9581416de1c425b8264e2cadd201eb709ec1584c381f3e98f51c1/httptools-0.6.4-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for watchfiles>=0.13 from https://files.pythonhosted.org/packages/bd/04/8c18986b79d106a88f54629f8f901cd725d76227c9a9191ada8ce8c962e8/watchfiles-1.0.0-cp311-none-win_amd64.whl.metadata
  Obtaining dependency information for websockets>=10.4 from https://files.pythonhosted.org/packages/bc/ef/2d81679acbe7057ffe2308d422f744497b52009ea8bab34b

In [7]:
pip install python-multipart

  Obtaining dependency information for python-multipart from https://files.pythonhosted.org/packages/e1/f4/ddd0fcdc454cf3870153ae16a818256523d31c3c8136e216bc6836ed4cd1/python_multipart-0.0.19-py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install opencv-python-headless

  Obtaining dependency information for opencv-python-headless from https://files.pythonhosted.org/packages/26/d0/22f68eb23eea053a31655960f133c0be9726c6a881547e6e9e7e2a946c4f/opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB 660.6 kB/s eta 0:00:59
   ---------------------------------------- 0.1/38.8 MB 1.3 MB/s eta 0:00:30
   ---------------------------------------- 0.3/38.8 MB 2.0 MB/s eta 0:00:20
   ---------------------------------------- 0.3/38.8 MB 2.2 MB/s eta 0:00:18
    --------------------------------------- 0.5/38.8 MB 2.4 MB/s eta 0:00:16
    --------------------------------------- 0.7/38.8 MB 2.6 MB/s eta 0:00:15
    --------------------------------------- 0.8/38.8 MB 2.8 MB/s eta 0:00:14
    --------------------------------------- 0.9/38.8 MB 2.7 MB/s eta 0:00:14
   - -------------------------------------- 1.1/38.8 MB 2.9 MB/s eta 0:0

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\Users\\DELA FARAHITA ZEIN\\anaconda3\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.



In [ ]:
import os
import cv2
import numpy as np
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
import tempfile
import shutil

app = FastAPI()

# Load Haar Cascade Classifier
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')

# Load pre-trained face recognition model
model = cv2.face.LBPHFaceRecognizer_create()
model.read("lbph_model.yml")

# Load labels
# labels = ["Dela", "Kinata", "Mulki", "Pascalis"]  
labels = ["Colin_Powell", "Donald_Rumsfeld", "George_W_Bush", "George_W_Bush", "Gerhard_Schroeder"]  

def detect_face(img):
    """
    Detect and crop face from an image
    
    Args:
        img (numpy.ndarray): Input image
    
    Returns:
        numpy.ndarray or None: Processed face image
    """
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    # If no face found, return None
    if len(faces) == 0:
        return None
    
    # Take the first detected face
    x, y, w, h = faces[0]
    
    # Crop and resize the face
    face_img = gray[y:y+h, x:x+w]
    face_img = cv2.resize(face_img, (100, 100))
    
    return face_img

@app.post("/recognize/")
async def recognize_face(file: UploadFile = File(...)):
    try:
        # Create a temporary file to save the uploaded image
        with tempfile.NamedTemporaryFile(delete=False, suffix=".jpg") as temp_file:
            shutil.copyfileobj(file.file, temp_file)
            temp_file_path = temp_file.name
        
        # Read the image
        img = cv2.imread(temp_file_path)
        
        # Remove the temporary file
        os.unlink(temp_file_path)
        
        # Detect and preprocess face
        face_img = detect_face(img)
        
        # If no face detected
        if face_img is None:
            return JSONResponse(content={
                "error": "No face detected in the image"
            }, status_code=400)
        
        # Predict using the model
        idx, confidence = model.predict(face_img)
        
        # Return prediction results
        return JSONResponse(content={
            "predicted_name": labels[idx],
            "confidence": float(confidence)
        })
    
    except Exception as e:
        return JSONResponse(content={
            "error": str(e)
        }, status_code=500)

# Optional: Health check endpoint
@app.get("/")
async def health_check():
    return {"status": "Face Recognition API is running"}